In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!ls "/content/drive/MyDrive/DeepLearning/Common Literacy"

Common_Literacy_AVGW2V_POS.ipynb
CommonLiteracyBertModels.ipynb
Common_Literacy_BOW_TFIDF_AvgW2V100D.ipynb
Common_Literacy_BOW_TFIDF_AvgW2V100D_SVD.ipynb
Common_Literacy_BOW_TFIDF_AVGW2V300D.ipynb
CommonLiteracy_CNN_Model.ipynb
Dataset


In [3]:
train ='/content/drive/MyDrive/DeepLearning/Common Literacy/Dataset/train.csv'
test = '/content/drive/MyDrive/DeepLearning/Common Literacy/Dataset/test.csv'

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from tqdm import tqdm

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import LayerNormalization, BatchNormalization
from keras.layers import Embedding, LSTM
from keras.layers import Conv1D, Flatten, MaxPool1D
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

%matplotlib inline

In [5]:
df_train = pd.read_csv(train)
df_test = pd.read_csv(test)

### NLTK Stopwords Removal

In [6]:
# importing stopwords from nltk library
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stopword = stopwords.words('english')
print(stopword)

stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'ag

### Cleaning the text data for any special characters and numerical characters

In [7]:
def testpreprocess(phrase):
    phrase = re.sub('[^A-Za-z0-9]+', ' ', phrase)
    phrase = re.sub(r'\w*\d\w*', '', phrase).strip()
    return phrase

In [8]:
preprocessed_excerpt = []
for sentence in tqdm(df_train['excerpt'].values):
  sent = testpreprocess(sentence)
  sent = sent.lower().strip()
  sent = ' '.join(stemmer.stem(i) for i in sent.split(" "))
  sent = ' '.join(e for e in sent.split(" ") if e not in stopword)
  preprocessed_excerpt.append(sent)

df_train['cleaned_excerpt'] = preprocessed_excerpt

100%|██████████| 2834/2834 [00:11<00:00, 250.24it/s]


### Pretrained Glove Vector assignment

In [9]:
# Refer this for step by step method https://stackoverflow.com/questions/50060241/how-to-use-glove-word-embeddings-file-on-google-colaboratory

!wget http://nlp.stanford.edu/data/glove.6B.zip # get the glove vector from stanford library

--2021-07-09 12:53:22--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-07-09 12:53:22--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-07-09 12:53:23--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [10]:
#unzip the downloaded glove vectors to drive
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [12]:
# # creating the dictionary if word and vectors
print('Indexing word vectors.')

d_model = 100
glove_dict = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    glove_dict[word] = coefs
f.close()

print('Found %s word vectors.' % len(glove_dict))

# checking the dictionary word shape
print(glove_dict['the'].shape)

Indexing word vectors.
Found 400000 word vectors.
(100,)


In [13]:
# creating a set of the keys from glove dictionary
glove_words = set(glove_dict.keys())

### Positional Encoding

In [14]:
import math
def positionalEncoding1DSine(max_sentence_length):
  positional_embeddings_sin = np.zeros(max_sentence_length)
  for position in range(max_sentence_length):
      positional_embeddings_sin[position] = (np.sin(position/(math.log(max_sentence_length))))
  return positional_embeddings_sin

def positionalEncoding1DCosine(max_sentence_length):
  positional_embeddings_cos = np.zeros(max_sentence_length)
  for position in range(max_sentence_length):
      positional_embeddings_cos[position] = (np.cos(position/(math.log(max_sentence_length))))
  return positional_embeddings_cos

def positionalEncoding1D(max_sentence_length):
  positional_embeddings_cos = np.zeros(max_sentence_length)
  positional_embeddings_sin = np.zeros(max_sentence_length)
  for position in range(max_sentence_length):
      positional_embeddings_cos[position] = (np.cos(position/(math.log(max_sentence_length))))
      positional_embeddings_sin[position] = (np.sin(position/(math.log(max_sentence_length))))
  positional_embeddings = np.add(positional_embeddings_cos, positional_embeddings_sin)
  return positional_embeddings

def positionalEncoding2d(max_sentence_length,d_model):
  positional_embeddings = np.zeros((max_sentence_length, d_model))
  for position in range(max_sentence_length):
      for i in range(0, d_model, 2):
        positional_embeddings[position, i] = (np.sin(position/(10000 ** ((2*i) / d_model))))
        positional_embeddings[position, i + 1] = (np.cos(position/(10000 ** ((2*(i+1)) / d_model))))
  return positional_embeddings

### Glove Average Word 2 Vec With Positional Encoding

In [23]:
def Avg_W2V_Pos(selfie): # give a pandas series into
  array_col_size = selfie.size
  AvgW2V_list = []; 
  for sentence in tqdm(selfie.values): 
      sentence = sentence.split()
      max_sentence_length = len(sentence)
      empty_vector = np.empty((0,d_model),dtype=float)
      posEmbed = positionalEncoding2d(max_sentence_length,d_model)
      for word in sentence:
          if word in glove_words: 
              vector = glove_dict[word] 
              vectorTranspose = np.reshape(vector,(-1,d_model))
              empty_vector = np.append(empty_vector,vectorTranspose,axis=0)
          else:
            vectorZeros = np.zeros((1,d_model),dtype=float)
            empty_vector = np.append(empty_vector,vectorZeros,axis=0)
      WordEmbedPOS = np.add(posEmbed, empty_vector)
      AvgW2V_list.append(WordEmbedPOS)
  # AvgW2V_Array = np.stack(AvgW2V_list, axis=0) # used to flatten the vector to one axis and convert the list of list to array
  return AvgW2V_list

### Splitting data into Train and cross validation(or test): Stratified Sampling

In [17]:
Y = df_train['target']
X = df_train['cleaned_excerpt']

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)

In [19]:
print(X_train[0:3])

774     neuron cell contain within brain spinal cord s...
2659    four day platt yet buffalo last year sign prov...
2723    thi whole glitter armi head king great royal s...
Name: cleaned_excerpt, dtype: object


### TF-IDF processing of text

In [20]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1),min_df=5)
vectorizer.fit(X_train)
feature_names = vectorizer.get_feature_names()

X_train_excerpt_tfidf = vectorizer.transform(X_train)
X_test_excerpt_tfidf = vectorizer.transform(X_test)

print("="*100)
print("After vectorizations")
print("="*50)
print(X_train_excerpt_tfidf.shape, Y_train.shape)
print(X_test_excerpt_tfidf.shape, Y_test.shape)
print("="*100)

After vectorizations
(1898, 3893) (1898,)
(936, 3893) (936,)


In [ ]:
tfidf_dict = dict(zip(vectorizer.get_feature_names(),list(vectorizer.idf_)))

### Word to Vector Representation

In [25]:
X_train_excerpt_w2v = Avg_W2V_Pos(X_train)
X_test_excerpt_w2v = Avg_W2V_Pos(X_test)

100%|██████████| 936/936 [00:13<00:00, 70.07it/s]


In [34]:
# adding padding to the text sequences
train_data = pad_sequences(X_train_excerpt_w2v, maxlen=200)
test_data = pad_sequences(X_test_excerpt_w2v,maxlen=200)

In [36]:
print(train_data.shape)
print(test_data.shape)

(1898, 200, 100)
(936, 200, 100)


### Tokenizer

In [ ]:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(X_train)

# train_sequences = tokenizer.texts_to_sequences(X_train)
# test_sequences = tokenizer.texts_to_sequences(X_test)

In [ ]:
# word_dict = tokenizer.word_index
# print(word_dict)

# # Inversing the document 
# word_dict_inv = {v: k for k, v in word_dict.items()}
# print(word_dict_inv)

{'wa': 1, 'hi': 2, 'thi': 3, 'one': 4, 'would': 5, 'veri': 6, 'littl': 7, 'said': 8, 'time': 9, 'use': 10, 'ha': 11, 'like': 12, 'could': 13, 'call': 14, 'two': 15, 'day': 16, 'make': 17, 'peopl': 18, 'water': 19, 'made': 20, 'go': 21, 'look': 22, 'way': 23, 'come': 24, 'mani': 25, 'great': 26, 'first': 27, 'onli': 28, 'see': 29, 'year': 30, 'old': 31, 'came': 32, 'also': 33, 'good': 34, 'work': 35, 'went': 36, 'get': 37, 'long': 38, 'thing': 39, 'differ': 40, 'man': 41, 'much': 42, 'live': 43, 'place': 44, 'ani': 45, 'well': 46, 'befor': 47, 'hous': 48, 'know': 49, 'even': 50, 'upon': 51, 'everi': 52, 'new': 53, 'take': 54, 'becaus': 55, 'may': 56, 'back': 57, 'away': 58, 'found': 59, 'state': 60, 'say': 61, 'part': 62, 'think': 63, 'mother': 64, 'thought': 65, 'home': 66, 'tree': 67, 'boy': 68, 'must': 69, 'us': 70, 'help': 71, 'put': 72, 'hand': 73, 'want': 74, 'three': 75, 'countri': 76, 'mr': 77, 'head': 78, 'give': 79, 'form': 80, 'still': 81, 'anoth': 82, 'world': 83, 'play': 84

In [37]:
# for k,v in tfidf_dict.items():
#   print(k)

In [ ]:
# sentencelength = []
# for i in train_sequences:
#   sentencelength.append(len(i))
# print(max(sentencelength))

138


In [ ]:
# sentencelength = []
# for i in test_sequences:
#   sentencelength.append(len(i))
# print(max(sentencelength))

128


In [ ]:
# train_seq_pos = []
# for i in train_sequences:
#   max_length = len(i)
#   i = np.array(i)
#   posSineEncod_train = positionalEncoding1DSine(max_length)
#   WordEmbedPOS_train = np.add(i, posSineEncod_train)
#   train_seq_pos.append(WordEmbedPOS_train)

In [ ]:
# test_seq_pos = []
# for i in test_sequences:
#   max_length = len(i)
#   i = np.array(i)
#   posSineEncod_test = positionalEncoding1DSine(max_length)
#   WordEmbedPOS_test = np.add(i, posSineEncod_test)
#   test_seq_pos.append(WordEmbedPOS_test)

In [ ]:
# print(train_seq_pos[0])

[5.70000000e+02 4.13323245e+03 7.54521625e+01 1.64647106e+02
 2.35180660e+03 6.54921906e+02 2.88986707e+02 5.86997455e+02
 2.49395356e+03 1.75385742e+03 8.71714319e+02 7.05320809e+01
 7.53206941e+01 6.10091739e+02 5.13857758e+02 6.00631569e+02
 7.44255643e+01 3.34251028e+02 2.23117523e+02 1.06803236e+03
 8.00021022e+00 1.02022830e+02 8.13098981e+02 3.12922449e+03
 3.92488142e-01 3.39559376e+03 8.17296085e-01 8.45605084e+03
 4.65728159e+03 1.03496921e+02 8.14685027e+02 9.08356060e+01
 2.00940408e+02 8.97993693e+02 2.14992540e+02 7.51937014e+02
 8.14830156e+02 6.01677819e+02 3.35488350e+02 9.27212813e+00
 3.13104100e+03 2.88980762e+03 8.13584785e+02 3.12938469e+03
 9.13218312e+02 1.17094753e+02 6.00020785e+02 3.34000461e+02
 2.11603489e+03 8.45612220e+03 2.13257590e+02 2.04336535e+01
 6.60640743e+02 1.04186751e+03 1.61310154e+03 1.68433001e+03
 2.35254039e+03 1.37372118e+03 2.08624515e+01 1.04295648e+03
 3.39799811e+03 6.61985059e+02 3.13191805e+03 6.62800739e+02
 1.68563957e+03 8.154433

In [ ]:
# adding padding to the text sequences
train_data = pad_sequences(train_sequences, maxlen=150)
test_data = pad_sequences(test_sequences,maxlen=150)

In [ ]:
# print(train_data[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0  570 4133   75  164 2351
  654  288  586 2493 1753  871   70   75  610  514  601   75  335  224
 1069    9  103  814 3130    1 3396    1 8456 4657  103  814   90  200
  897  214  751  814  601  335    9 3131 2890  814 3130  914  118  601
  335 2117 8457  214   21  661 1042 1613 1684 2352 1373   20 1042 3397
  661 3131  662 1685  815 2684    9  993  335 4658]


In [ ]:
# # adding padding to the text sequences
# train_data_pos = pad_sequences(train_seq_pos, maxlen=150)
# test_data_pos = pad_sequences(test_seq_pos, maxlen=150)

In [ ]:
# print(train_data.shape)
# print(test_data.shape)

(1898, 150)
(936, 150)


In [38]:
# Convert target to array
Y_train = np.asarray(Y_train)
Y_test = np.asarray(Y_test)

In [39]:
print(Y_train.shape)

(1898,)


In [ ]:
# # create a weight matrix for words in training docs
# embedding_matrix = np.zeros((25000, 100))
# for word, index in tokenizer.word_index.items():
#     if index > 25000 - 1:
#         break
#     else:
#         embedding_vector = glove_dict.get(word)
#         if embedding_vector is not None:
#             embedding_matrix[index] = embedding_vector

In [ ]:
# print(embedding_matrix.shape)

(25000, 100)


### Model Definition

In [ ]:
Vocab_Size = 25000 # this is the total number of words being sent
Word_Embedding_Size = 100 # this value is coming from Glove Vector Size
Input_Sequence = 150 # this is based on the padding we have given

In [ ]:
regressor = Sequential()

# Step 1 Adding an embedding layer
regressor.add(Embedding(Vocab_Size, Word_Embedding_Size, input_length=Input_Sequence, weights=[embedding_matrix], trainable=False))

# Step 2 Adding a first convolutional layer with Max pooling
regressor.add(Conv1D(64, 5, padding = 'same', activation = 'relu')),
regressor.add(MaxPool1D(pool_size=5))
regressor.add(Dropout(0.5))

# Step 3 Adding a first convolutional layer with Max pooling
regressor.add(Conv1D(32, 5, padding = 'same', activation = 'relu')),
regressor.add(MaxPool1D(pool_size=5))
regressor.add(Dropout(0.5))

regressor.add(LSTM(250))

# Step 5 - Full connection layer with sigmoid
regressor.add(Dense(256, activation = 'relu'))
regressor.add(Dense(128, activation = 'relu'))
regressor.add(Dense(1, activation = 'linear'))

# Compiling the CNN
regressor.compile(optimizer = 'adam', loss = 'mse', metrics=['mse'])
regressor.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 100)          2500000   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 150, 64)           32064     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 30, 64)            0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 30, 64)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 30, 32)            10272     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 6, 32)             0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 6, 32)            

In [ ]:
regressor.fit(train_data, Y_train, batch_size=200, epochs=100, validation_split=0.2, verbose=2)

Epoch 1/100
8/8 - 4s - loss: 1.2686 - mse: 1.2686 - val_loss: 1.1800 - val_mse: 1.1800
Epoch 2/100
8/8 - 1s - loss: 1.0171 - mse: 1.0171 - val_loss: 1.1120 - val_mse: 1.1120
Epoch 3/100
8/8 - 1s - loss: 0.9132 - mse: 0.9132 - val_loss: 0.9775 - val_mse: 0.9775
Epoch 4/100
8/8 - 1s - loss: 0.7910 - mse: 0.7910 - val_loss: 0.9294 - val_mse: 0.9294
Epoch 5/100
8/8 - 1s - loss: 0.7394 - mse: 0.7394 - val_loss: 0.8280 - val_mse: 0.8280
Epoch 6/100
8/8 - 1s - loss: 0.6681 - mse: 0.6681 - val_loss: 0.8037 - val_mse: 0.8037
Epoch 7/100
8/8 - 1s - loss: 0.6206 - mse: 0.6206 - val_loss: 0.7884 - val_mse: 0.7884
Epoch 8/100
8/8 - 1s - loss: 0.6055 - mse: 0.6055 - val_loss: 0.7457 - val_mse: 0.7457
Epoch 9/100
8/8 - 1s - loss: 0.6039 - mse: 0.6039 - val_loss: 0.7452 - val_mse: 0.7452
Epoch 10/100
8/8 - 1s - loss: 0.5342 - mse: 0.5342 - val_loss: 0.7061 - val_mse: 0.7061
Epoch 11/100
8/8 - 1s - loss: 0.5046 - mse: 0.5046 - val_loss: 0.7819 - val_mse: 0.7819
Epoch 12/100
8/8 - 1s - loss: 0.4981 - ms

In [ ]:
from sklearn import metrics

Y_pred_test = regressor.predict(test_data)
test_residuals = Y_test - Y_pred_test
mse_test = metrics.mean_squared_error(Y_test,Y_pred_test)

In [ ]:
print("Root Mean squared Error Test:",np.sqrt(mse_test))

Root Mean squared Error Test: 0.9360322721552319


### Positional Embedding 1 Dimension

In [65]:
train_data_reshape1 = np.expand_dims(train_data, axis=1)
test_data_reshape1 = np.expand_dims(test_data, axis=1)

In [62]:
print(train_data_reshape1.shape)

(1898, 1, 200, 100)


In [41]:
from keras.layers import Conv2D
from keras.layers import MaxPool2D

In [90]:
classifier = Sequential()

# Adding a first convolutional layer with Max pooling
classifier.add(Conv1D(300, 5, padding = 'same', activation = 'relu', input_shape = (200, 100))),
classifier.add(MaxPool1D(pool_size=5))
classifier.add(Dropout(0.8))

# Step 3 Adding a first convolutional layer with Max pooling
classifier.add(Conv1D(200, 5, padding = 'same', activation = 'relu')),
classifier.add(Conv1D(100, 3, padding = 'same', activation = 'relu')),
classifier.add(Conv1D(100, 1, padding = 'same', activation = 'relu')),
classifier.add(MaxPool1D(pool_size=5))
classifier.add(Dropout(0.5))

classifier.add(LSTM(200))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 5 - Full connection layer with sigmoid
classifier.add(Dense(256, activation = 'relu'))
classifier.add(Dense(128, activation = 'relu'))
classifier.add(Dense(64, activation = 'relu'))
classifier.add(Dense(1, activation = 'linear'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'mse', metrics=['mse'])
classifier.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_39 (Conv1D)           (None, 200, 300)          150300    
_________________________________________________________________
max_pooling1d_28 (MaxPooling (None, 40, 300)           0         
_________________________________________________________________
dropout_30 (Dropout)         (None, 40, 300)           0         
_________________________________________________________________
conv1d_40 (Conv1D)           (None, 40, 200)           300200    
_________________________________________________________________
conv1d_41 (Conv1D)           (None, 40, 100)           60100     
_________________________________________________________________
conv1d_42 (Conv1D)           (None, 40, 100)           10100     
_________________________________________________________________
max_pooling1d_29 (MaxPooling (None, 8, 100)          

In [91]:
classifier.fit(train_data, Y_train, batch_size=200, epochs=50, validation_split=0.2, verbose=2)

Epoch 1/50
8/8 - 10s - loss: 1.3310 - mse: 1.3310 - val_loss: 1.0170 - val_mse: 1.0170
Epoch 2/50
8/8 - 7s - loss: 1.0978 - mse: 1.0978 - val_loss: 1.0170 - val_mse: 1.0170
Epoch 3/50
8/8 - 7s - loss: 1.0738 - mse: 1.0738 - val_loss: 1.1562 - val_mse: 1.1562
Epoch 4/50
8/8 - 7s - loss: 1.0671 - mse: 1.0671 - val_loss: 1.0519 - val_mse: 1.0519
Epoch 5/50
8/8 - 7s - loss: 1.0527 - mse: 1.0527 - val_loss: 1.0342 - val_mse: 1.0342
Epoch 6/50
8/8 - 7s - loss: 1.0492 - mse: 1.0492 - val_loss: 1.0283 - val_mse: 1.0283
Epoch 7/50
8/8 - 7s - loss: 1.0363 - mse: 1.0363 - val_loss: 0.9991 - val_mse: 0.9991
Epoch 8/50
8/8 - 7s - loss: 1.0552 - mse: 1.0552 - val_loss: 0.9831 - val_mse: 0.9831
Epoch 9/50
8/8 - 7s - loss: 1.0125 - mse: 1.0125 - val_loss: 0.9990 - val_mse: 0.9990
Epoch 10/50
8/8 - 7s - loss: 0.9344 - mse: 0.9344 - val_loss: 0.7544 - val_mse: 0.7544
Epoch 11/50
8/8 - 7s - loss: 0.8627 - mse: 0.8627 - val_loss: 1.0399 - val_mse: 1.0399
Epoch 12/50
8/8 - 7s - loss: 0.8122 - mse: 0.8122 -